In [77]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler

def seller_group_transform(seller_data):
  # Check for constant values (all identical)
  if seller_data['MaxPotential'].nunique() == 1:
    # Handle constant values (e.g., assign specific group)
    seller_data['Group'] = 'Constant'
  else:
    try:
      # Apply Box-Cox transformation (if possible)
      transformed_data, lambda_value = stats.boxcox(seller_data['MaxPotential'])
      seller_data['Transformed MaxPotential'] = transformed_data_full = stats.boxcox(seller_data['MaxPotential'], lmbda=lambda_value)

      # Define conditions for grouping
      conditions = [
          (seller_data['Transformed MaxPotential'] < (seller_data['Transformed MaxPotential'].median())),
          (seller_data['Transformed MaxPotential'] >= (seller_data['Transformed MaxPotential'].median())) & (seller_data['Transformed MaxPotential'] < (seller_data['Transformed MaxPotential'].median()+(1.25*(seller_data['Transformed MaxPotential'].std())))),
          (seller_data['Transformed MaxPotential'] >= (seller_data['Transformed MaxPotential'].median()+(1.25*(seller_data['Transformed MaxPotential'].std())))) & (seller_data['Transformed MaxPotential'] < (seller_data['Transformed MaxPotential'].median()+(2.125*(seller_data['Transformed MaxPotential'].std())))),
          (seller_data['Transformed MaxPotential'] >= (seller_data['Transformed MaxPotential'].median()+(2.125*(seller_data['Transformed MaxPotential'].std()))))]

      # Define labels for groups
      values = ['Low','Medium','High','Outlier']

      # Add group labels to the DataFrame
      seller_data['Group'] = np.select(conditions, values, default='Other')
    except ValueError:
      # If Box-Cox fails (e.g., due to negative values), apply alternative transformation
      scaler = StandardScaler()
      seller_data['Transformed MaxPotential'] = scaler.fit_transform(seller_data[['MaxPotential']])[:, 0]
      # Group based on the scaled data (consider adapting conditions if needed)

  return seller_data

# Read data from CSV
data = pd.read_csv("SellerTPOWorking_Last12Months.csv")

# Group data by seller and apply transformation and grouping
data_grouped = data.groupby('SellerId').apply(seller_group_transform)


c:\Users\Akhil\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:935: RuntimeWarning: divide by zero encountered in log
  return (lmb - 1) * np.sum(logdata, axis=0) - N/2 * np.log(variance)
c:\Users\Akhil\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_optimize.py:2884: RuntimeWarning: invalid value encountered in scalar divide
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom
c:\Users\Akhil\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_optimize.py:2416: RuntimeWarning: invalid value encountered in scalar subtract
  tmp1 = (x - w) * (fx - fv)
c:\Users\Akhil\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_optimize.py:2417: RuntimeWarning: invalid value encountered in scalar subtract
  tmp2 = (x - v) * (fx - fw)
c:\Users\Akhil\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:935: RuntimeWarning: divide by zero encountered in log
  ret

                   SellerId         SellerName     BuyerId  \
SellerId                                                     
1476624891 878   1476624891  KIK INTERNATIONAL  1491052771   
           965   1476624891  KIK INTERNATIONAL  1490519253   
           1099  1476624891  KIK INTERNATIONAL  1490888760   
           2162  1476624891  KIK INTERNATIONAL  1491175107   
           3056  1476624891  KIK INTERNATIONAL  1490480352   

                                  BuyerName  MaxMonthlyGMV  MinMonthlyGMV  \
SellerId                                                                    
1476624891 878          ARAFA SHOPPING MALL        82293.0        33374.0   
           965   ROJINA FOOTWEAR BHID BAZAR        66351.0        44138.0   
           1099          NEW PAPPU FOOTWEAR        52723.0        52723.0   
           2162                  STYLE WALK        39260.0        39260.0   
           3056               AARON TRADERS        49389.0        18231.0   

                 Last6Mon

In [80]:
data_grouped.head()

SellerId         SellerName     BuyerId  \
SellerId                                                     
1476624891 878   1476624891  KIK INTERNATIONAL  1491052771   
           965   1476624891  KIK INTERNATIONAL  1490519253   
           1099  1476624891  KIK INTERNATIONAL  1490888760   
           2162  1476624891  KIK INTERNATIONAL  1491175107   
           3056  1476624891  KIK INTERNATIONAL  1490480352   

                                  BuyerName  MaxMonthlyGMV  MinMonthlyGMV  \
SellerId                                                                    
1476624891 878          ARAFA SHOPPING MALL        82293.0        33374.0   
           965   ROJINA FOOTWEAR BHID BAZAR        66351.0        44138.0   
           1099          NEW PAPPU FOOTWEAR        52723.0        52723.0   
           2162                  STYLE WALK        39260.0        39260.0   
           3056               AARON TRADERS        49389.0        18231.0   

                 Last6MonthGMV  Last3MonthGMV  LastMonthGMV  AvgMonthlyGMV  \
SellerId                                                                     
1476624891 878             0.0            0.0           0.0        57834.0   
           965         55244.0        55244.0       66351.0        55244.0   
           1099            0.0            0.0           0.0        52723.0   
           2162        39260.0            0.0           0.0        39260.0   
           3056            0.0            0.0           0.0        33810.0   

                 MaxPotential         BuyerBucket BuyerType  \
SellerId                                                      
1476624891 878        57834.0      Promoter Buyer     Churn   
           965        66351.0      Promoter Buyer  WorkBase   
           1099       52723.0  Non Promoter Buyer     Churn   
           2162       39260.0      Promoter Buyer  WorkBase   
           3056       33810.0      Promoter Buyer  WorkBase   

                 Transformed MaxPotential    Group  
SellerId                                            
1476624891 878                   1.559566  Outlier  
           965                   1.559683  Outlier  
           1099                  1.559482  Outlier  
           2162                  1.559175  Outlier  
           3056                  1.558996  Outlier

In [79]:
data_grouped.to_csv('Akhil_Check.csv',index=False)